# Qiskit Implementation
"https://learn.qiskit.org/course/ch-algorithms/shors-algorithm"
In this example we will solve the period finding problem N=15 with a=7. One can check that r=4.

In [ ]:

import matplotlib.pyplot as plt
import numpy as np
from qiskit import QuantumCircuit,transpile
from qiskit.visualization import plot_histogram
from qiskit_aer import Aer              # used to make simulations (without real quantum computer, with or without noise)
from math import gcd
from numpy.random import randint
import pandas as pd
from fractions import Fraction
print("Imports Successful")

Below is the circuit for U|x> = |a x mod 15> (see https://arxiv.org/abs/quant-ph/0205095) (he dit not explained in class)

In [ ]:

def c_amod15(a, power):
    """Controlled multiplication by a mod 15"""
    if a not in [2,4,7,8,11,13]:
        raise ValueError("'a' must be 2,4,7,8,11 or 13")
    U = QuantumCircuit(4)
    for _iteration in range(power):
        if a in [2,13]:
            U.swap(2,3)
            U.swap(1,2)
            U.swap(0,1)
        if a in [7,8]:
            U.swap(0,1)
            U.swap(1,2)
            U.swap(2,3)
        if a in [4, 11]:
            U.swap(1,3)
            U.swap(0,2)
        if a in [7,11,13]:
            for q in range(4):
                U.x(q)
    U = U.to_gate()
    U.name = f"{a}^{power} mod 15"
    c_U = U.control()
    return c_U

We will use L=4 qubits for the second registry such that 15 <= 2^L , and 8 counting qubits (first registry): minimum 2L.

In [ ]:
# Specify variables
N_COUNT = 8  # number of counting qubits 
a = 7

#We also import the circuit for the QFT
def qft_dagger(n):
    """n-qubit QFTdagger the first n qubits in circ"""
    qc = QuantumCircuit(n)
    # Don't forget the Swaps!
    for qubit in range(n//2):
        qc.swap(qubit, n-qubit-1)
    for j in range(n):
        for m in range(j):
            qc.cp(-np.pi/float(2**(j-m)), m, j)
        qc.h(j)
    qc.name = "QFT†"
    return qc

With these building blocks we can easily construct the circuit for Shor's algorithm:

In [ ]:
# Create QuantumCircuit with N_COUNT counting qubits
# plus L=4 qubits for U to act on
qc = QuantumCircuit(N_COUNT + 4, N_COUNT)

# Initialize counting qubits
# in state |+>
for q in range(N_COUNT):
    qc.h(q)

# And auxiliary register in state |1>
qc.x(N_COUNT)

# Do controlled-U operations
for q in range(N_COUNT):
    qc.append(c_amod15(a, 2**q),
             [q] + [i+N_COUNT for i in range(4)])

# Do inverse-QFT
qc.append(qft_dagger(N_COUNT), range(N_COUNT))

# Measure circuit
qc.measure(range(N_COUNT), range(N_COUNT))
display(qc.draw('mpl',fold=-1))

Let's see what results we measure:

In [ ]:
aer_sim = Aer.get_backend('aer_simulator')  # set the simulator, it could aslo be IBMQasmSimulator for example
t_qc = transpile(qc, aer_sim)
counts = aer_sim.run(t_qc).result().get_counts()
plot_histogram(counts)

Since we have 8 qubits, these results correspond to measured phases of:

In [ ]:
rows, measured_phases = [], []
for output in counts:
    decimal = int(output, 2)  # Convert (base 2) string to decimal -> k
    phase = decimal/(2**N_COUNT)  # Find corresponding eigenvalue -> theta=k/2^t
    measured_phases.append(phase)
    # Add these values to the rows in our table:
    rows.append([f"{output}(bin) = {decimal:>3}(dec)",
                 f"{decimal}/{2**N_COUNT} = {phase:.2f}"])
# Print the rows in a table
headers=["Register Output", "Phase"]
df = pd.DataFrame(rows, columns=headers)
print(df)

#We can now use the continued fractions algorithm to attempt to find s and r. Python has this functionality built in: We can use the fractions module to turn a float into a Fraction object, for example:"
Fraction(0.666)

#Because this gives fractions that return the result exactly (in this case, 0.6660000...), this can give gnarly results like the one above. We can use the .limit_denominator() method to get the fraction that most closely resembles our float, with denominator below a certain value:"

# Get fraction that most closely resembles 0.666
# with denominator < 15
Fraction(0.666).limit_denominator(15)

#Much nicer! The order (r) must be less than N, so we will set the maximum denominator to be 15:"

rows = []
for phase in measured_phases:
    frac = Fraction(phase).limit_denominator(15)
    rows.append([phase,
                 f"{frac.numerator}/{frac.denominator}",
                 frac.denominator])
# Print as a table
headers=["Phase", "Fraction", "Guess for r"]
df = pd.DataFrame(rows, columns=headers)
print(df)

#We can see that two of the measured eigenvalues provided us with the correct result: r=4, and we can see that Shor’s algorithm has a chance of failing. These bad results are because s=0, or because and are not coprime and instead of r we are given a factor of r. The easiest solution to this is to simply repeat the experiment until we get a satisfying result for r."